In [1]:
import pandas as pd
import numpy as np

### 1. 데이터 불러오기

In [2]:
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0


In [3]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
dtype: object

In [4]:
selloutData["REGIONID"] = selloutData["REGIONID"].astype(str)
selloutData["PRODUCT"] = selloutData["PRODUCT"].astype(str)
selloutData["YEARWEEK"] = selloutData["YEARWEEK"].astype(str)
selloutData["QTY"] = selloutData["QTY"].astype(float)
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK     object
QTY         float64
dtype: object

In [5]:
selloutData[["REGIONID", "PRODUCT"]].head()

,REGIONID,PRODUCT
0,A60,PRODUCT4
1,A60,PRODUCT59
2,A60,PRODUCT34
3,A60,PRODUCT47
4,A60,PRODUCT56


### QTY_NEW 컬럼 생성(QTY<0 > 0)

In [6]:
selloutData["QTY_NEW"] = np.where(selloutData["QTY"]< 0,0,selloutData["QTY"])

### YEAR, WEEK 컬럼을 생성,  WEEK  52 이하인 데이터 조회

In [7]:
selloutData["YEAR"] = selloutData["YEARWEEK"].str[0:4]
selloutData["WEEK"] = selloutData["YEARWEEK"].str[4:]
refinedSelloutData2 = selloutData
refinedSelloutData = selloutData[selloutData.WEEK.astype(int) <= 52]
refinedSelloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,23.0,2014,02


In [8]:
refinedSelloutData["WEEK"].max()

'52'

In [9]:
groupKey2 = ["REGIONID","PRODUCT"]

groupData2 = selloutData.\
groupby(groupKey2).agg\
(['mean','std'])["QTY_NEW"]
groupData2.head()

mean            std
REGIONID PRODUCT                                
A00      PRODUCT34     132.445860     154.867743
         PRODUCT58     105.235669     273.435108
         PRODUCT59   36086.547771   15192.174914
A01      PRODUCT1     5716.726115    3704.948589
         PRODUCT12  197052.777070  198866.717131

In [10]:
groupData3 = groupData2.reset_index()
groupData3.head()

,REGIONID,PRODUCT,mean,std
0,A00,PRODUCT34,132.445860,154.867743
1,A00,PRODUCT58,105.235669,273.435108
2,A00,PRODUCT59,36086.547771,15192.174914
3,A01,PRODUCT1,5716.726115,3704.948589
4,A01,PRODUCT12,197052.777070,198866.717131


In [11]:
groupKey3 = ["REGIONID","PRODUCT","YEAR"]

groupData3 = refinedSelloutData2.\
groupby(groupKey3).agg\
(['mean','std'])["QTY_NEW"]
groupData3.head()

mean         std
REGIONID PRODUCT   YEAR                        
A00      PRODUCT34 2014  275.961538  162.179634
                   2015   85.698113   72.858024
                   2016   36.576923   94.897539
         PRODUCT58 2014    2.673077    3.889257
                   2015    7.547170   14.909427

In [12]:
groupData3.columns = ["QTY_MEAN", "QTY_STD"]
groupData4 = groupData3.reset_index()
groupData4.head()

,REGIONID,PRODUCT,YEAR,QTY_MEAN,QTY_STD
0,A00,PRODUCT34,2014,275.961538,162.179634
1,A00,PRODUCT34,2015,85.698113,72.858024
2,A00,PRODUCT34,2016,36.576923,94.897539
3,A00,PRODUCT58,2014,2.673077,3.889257
4,A00,PRODUCT58,2015,7.547170,14.909427


In [37]:
mergeKey = ["REGIONID", "PRODUCT", "YEAR"]

In [52]:
mergedData = pd.merge(groupData3, refinedSelloutData,\
        left_on=mergeKey, right_on = mergeKey)\
[["REGIONID","PRODUCT","YEARWEEK","YEAR","WEEK","QTY_NEW","QTY_MEAN","QTY_STD"]]

sortkey = ["REGIONID","PRODUCT","YEARWEEK"]

mergedData= mergedData.sort_values(sortkey)
mergedData=mergedData.reset_index(drop = True)
mergedData.head()

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY_NEW,QTY_MEAN,QTY_STD
0,A00,PRODUCT34,201401,2014,01,661.0,275.961538,162.179634
1,A00,PRODUCT34,201402,2014,02,679.0,275.961538,162.179634
2,A00,PRODUCT34,201403,2014,03,578.0,275.961538,162.179634
3,A00,PRODUCT34,201404,2014,04,532.0,275.961538,162.179634
4,A00,PRODUCT34,201405,2014,05,516.0,275.961538,162.179634
